In [1]:
import numpy as np
import gurobi as gp
import pandas as pd

In [2]:
demand_df = pd.read_csv('demand_data.csv')
demand_df

,day,demand
0,1,2925
1,2,6273
2,3,11783
3,4,7924
4,5,1103
5,6,2208
6,7,371
7,8,6456
8,9,736
9,10,1269


In [3]:
nd = demand_df.shape[0] # number of days of demand data
nd

25

In [4]:
cost = 0.5
price = 1.25

In [6]:
# decision variables are (q, h1, h2, ..., h25)
obj = np.zeros(nd+1)
obj[1:] = 1.0/nd
lb = np.zeros(nd+1)
lb[1:] = -np.inf # quantity printed needs to be non-negative, but profit on a given day could possibly be negative
rhs = np.zeros(2*nd)
direction = np.array(['<']*(2*nd))

In [7]:
obj

array([0.  , 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04,
       0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04,
       0.04, 0.04, 0.04, 0.04])

In [8]:
A = np.zeros((2*nd,nd+1))
for r in range(nd):
    A[2*r,[0,r+1]] = [cost,1] # location 1 is q, location r+1 is this h
    rhs[2*r] = price*demand_df.demand[r]
    A[2*r+1,[0,r+1]] = [cost-price,1]
    rhs[2*r+1] = 0
A

array([[ 0.5 ,  1.  ,  0.  , ...,  0.  ,  0.  ,  0.  ],
       [-0.75,  1.  ,  0.  , ...,  0.  ,  0.  ,  0.  ],
       [ 0.5 ,  0.  ,  1.  , ...,  0.  ,  0.  ,  0.  ],
       ...,
       [-0.75,  0.  ,  0.  , ...,  0.  ,  1.  ,  0.  ],
       [ 0.5 ,  0.  ,  0.  , ...,  0.  ,  0.  ,  1.  ],
       [-0.75,  0.  ,  0.  , ...,  0.  ,  0.  ,  1.  ]])

In [26]:
spMod = gp.Model()
spMod_x = spMod.addMVar(len(obj),lb=lb) # tell the model how many variables there are
# spMod_x = spMod.addMVar(len(obj),lb=lb, vtype = ['I']+['C']*nd) # tell the model how many variables there are
# must define the variables before adding constraints because variables go into the constraints
spMod_con = spMod.addMConstrs(A, spMod_x, direction, rhs) # NAME THE CONSTRAINTS!!! so we can get information about them later!
spMod.setMObjective(None,obj,0,sense=gp.GRB.MAXIMIZE) # add the objective to the model...we'll talk about the None and the 0

spMod.Params.OutputFlag = 0 # tell gurobi to shut up!!
spMod.optimize()


In [27]:
spMod_x.x[0]

2678.0

In [28]:
spMod.objval

858.6500000000002

In [29]:
numsold = np.minimum(spMod_x.x[0],demand_df.demand) # on each day, how many did we actually sell
np.mean(price*numsold - cost*spMod_x.x[0]) # what was the average profit? exactly equal to the objective value!

858.65